In [68]:
import pandas as pd
import os
from glob import glob
import reports

In [98]:
parent_dirformat = '/media/dmlab/My Passport/DATA/cross-domain/test_{}'

In [66]:
target_mode = 'raw'
parent_dir = parent_dirformat.format(target_mode)
filepaths = glob(os.path.join(parent_dir, 'target=*_source=*_raw/classification_report.csv'))
unk_fileformat = os.path.join(parent_dir, 'target={}_source={}_unk/classification_report.csv')

records = []
for filepath in filepaths:
    target, source, mode = os.path.basename(os.path.dirname(filepath)).split('_')
    target, source = target.replace('target=',''), source.replace('source=', '')

    df = pd.read_csv(filepath)
    raw_acc = '{:.2f}'.format(100*df[(df['Unnamed: 0']=='accuracy')]['f1-score'].values[0])

    df = pd.read_csv(unk_fileformat.format(target, source))
    unk_acc = '{:.2f}'.format(100*df[(df['Unnamed: 0']=='accuracy')]['f1-score'].values[0])

    records.append(('{}->{}'.format(source, target), float(raw_acc), float(unk_acc)))

result_df = pd.DataFrame(records, columns = ['source->target', 'raw', 'unk'])
print(result_df.mean())
result_df

raw    82.140000
unk    81.118333
dtype: float64


,source->target,raw,unk
0,dvd->books,86.58,87.08
1,electronics->books,83.65,80.27
2,kitchen->books,83.10,81.74
3,books->dvd,83.88,83.38
4,electronics->dvd,81.95,78.85
5,kitchen->dvd,82.41,82.21
6,books->electronics,77.92,76.59
7,dvd->electronics,74.50,76.49
8,kitchen->electronics,83.53,81.90
9,books->kitchen,81.34,81.79


In [86]:
target_mode = 'unk'
parent_dir = parent_dirformat.format(target_mode)
filepaths = glob(os.path.join(parent_dir, 'target=*_source=*_raw/classification_report.csv'))
unk_fileformat = os.path.join(parent_dir, 'target={}_source={}_unk/classification_report.csv')

records = []
for filepath in filepaths:
    target, source, mode = os.path.basename(os.path.dirname(filepath)).split('_')
    target, source = target.replace('target=',''), source.replace('source=', '')

    df = pd.read_csv(filepath)
    df.set_index('Unnamed: 0', inplace=True)
    raw_acc = '{:.2f}'.format(100*df.filter(items = ['accuracy'], axis=0)['f1-score'].values[0])

    df = pd.read_csv(unk_fileformat.format(target, source))
    df.set_index('Unnamed: 0', inplace=True)
    unk_acc = '{:.2f}'.format(100*df.filter(items = ['accuracy'], axis=0)['f1-score'].values[0])

    records.append(('{}->{}'.format(source, target), float(raw_acc), float(unk_acc)))

result_df = pd.DataFrame(records, columns = ['source->target', 'raw', 'unk'])
print(result_df.mean())
result_df

raw    81.660833
unk    80.831667
dtype: float64


,source->target,raw,unk
0,dvd->books,85.47,85.92
1,electronics->books,83.70,80.37
2,kitchen->books,82.64,81.43
3,books->dvd,83.27,83.43
4,electronics->dvd,82.61,79.66
5,kitchen->dvd,81.65,82.21
6,books->electronics,76.95,76.03
7,dvd->electronics,74.09,76.34
8,kitchen->electronics,83.07,81.44
9,books->kitchen,80.53,80.63


using post-trained

In [99]:
target_mode = 'using_post-trained_raw'
parent_dir = parent_dirformat.format(target_mode)
filepaths = glob(os.path.join(parent_dir, 'target=*_source=*_raw/classification_report.csv'))
unk_fileformat = os.path.join(parent_dir, 'target={}_source={}_unk/classification_report.csv')

records = []
for filepath in filepaths:
    target, source, mode = os.path.basename(os.path.dirname(filepath)).split('_')
    target, source = target.replace('target=',''), source.replace('source=', '')

    df = pd.read_csv(filepath)
    raw_acc = '{:.2f}'.format(100*df[(df['Unnamed: 0']=='accuracy')]['f1-score'].values[0])

    df = pd.read_csv(unk_fileformat.format(target, source))
    unk_acc = '{:.2f}'.format(100*df[(df['Unnamed: 0']=='accuracy')]['f1-score'].values[0])

    records.append(('{}->{}'.format(source, target), float(raw_acc), float(unk_acc)))

result_df = pd.DataFrame(records, columns = ['source->target', 'raw', 'unk'])
print(result_df.mean())
result_df

raw    83.120833
unk    82.371667
dtype: float64


,source->target,raw,unk
0,dvd->books,85.42,85.37
1,electronics->books,84.41,81.99
2,kitchen->books,84.86,84.46
3,books->dvd,84.34,84.49
4,electronics->dvd,81.70,78.09
5,kitchen->dvd,84.65,84.19
6,books->electronics,78.58,80.11
7,dvd->electronics,75.68,80.06
8,kitchen->electronics,85.16,81.49
9,books->kitchen,83.56,82.30


upper limit
- 데이터가 작아서 학습이 안됨

In [94]:
filepaths = glob('/media/dmlab/My Passport/DATA/cross-domain/finetune_upper_limit/*_raw_k=1/classification_report.csv')
filepath_format = '/media/dmlab/My Passport/DATA/cross-domain/finetune_upper_limit/{}_{}_k={}/classification_report.csv'

records = []
for filepath in filepaths:
    domain = os.path.basename(os.path.dirname(filepath)).split('_')[0]
    raw_acc = '{:.2f}'.format(100*get_average_accuracy(domain, 'raw'))
    unk_acc = '{:.2f}'.format(100*get_average_accuracy(domain, 'unk'))

    records.append((domain, float(raw_acc), float(unk_acc)))

result_df = pd.DataFrame(records, columns = ['domain (kfold)', 'raw', 'unk'])
print(result_df.mean())
result_df

raw    79.5450
unk    78.7675
dtype: float64


,domain (kfold),raw,unk
0,books,77.75,81.79
1,dvd,82.46,76.81
2,electronics,75.57,78.43
3,kitchen,82.40,78.04


using post-trained models

In [95]:
target_mode = 'using_post-trained'
parent_dir = parent_dirformat.format(target_mode)
filepaths = glob(os.path.join(parent_dir, 'target=*_source=*_raw/classification_report.csv'))
unk_fileformat = os.path.join(parent_dir, 'target={}_source={}_unk/classification_report.csv')

records = []
for filepath in filepaths:
    target, source, mode = os.path.basename(os.path.dirname(filepath)).split('_')
    target, source = target.replace('target=',''), source.replace('source=', '')

    df = pd.read_csv(filepath)
    raw_acc = '{:.2f}'.format(100*df[(df['Unnamed: 0']=='accuracy')]['f1-score'].values[0])

    df = pd.read_csv(unk_fileformat.format(target, source))
    unk_acc = '{:.2f}'.format(100*df[(df['Unnamed: 0']=='accuracy')]['f1-score'].values[0])

    records.append(('{}->{}'.format(source, target), float(raw_acc), float(unk_acc)))

result_df = pd.DataFrame(records, columns = ['source->target', 'raw', 'unk'])
print(result_df.mean())
result_df

raw    85.825833
unk    85.762500
dtype: float64


,source->target,raw,unk
0,dvd->books,87.94,88.50
1,electronics->books,86.53,86.23
2,kitchen->books,82.85,82.90
3,books->dvd,87.70,87.39
4,electronics->dvd,85.97,85.87
5,kitchen->dvd,83.02,84.19
6,books->electronics,82.20,81.85
7,dvd->electronics,83.94,84.29
8,kitchen->electronics,87.25,85.57
9,books->kitchen,86.09,86.19


upper limit using post_trained models with 키워드[UNK]
- 데이터가 작아서 비교가 무의미한 듯

In [96]:
filepaths = glob('/media/dmlab/My Passport/DATA/cross-domain/finetune_upper_limit_using_post-trained_unk/*_raw_k=1/classification_report.csv')
filepath_format = '/media/dmlab/My Passport/DATA/cross-domain/finetune_upper_limit_using_post-trained_unk/{}_{}_k={}/classification_report.csv'

records = []
for filepath in filepaths:
    domain = os.path.basename(os.path.dirname(filepath)).split('_')[0]
    raw_acc = '{:.2f}'.format(100*get_average_accuracy(domain, 'raw'))
    unk_acc = '{:.2f}'.format(100*get_average_accuracy(domain, 'unk'))

    records.append((domain, float(raw_acc), float(unk_acc)))

result_df = pd.DataFrame(records, columns = ['domain (kfold)', 'raw', 'unk'])
print(result_df.mean())
result_df

raw    81.6725
unk    82.5725
dtype: float64


,domain (kfold),raw,unk
0,books,80.07,83.35
1,dvd,84.85,81.49
2,electronics,76.13,80.83
3,kitchen,85.64,84.62


upper limit using post_trained models with Raw
- 데이터가 작아서 비교가 무의미한 듯

In [97]:
filepaths = glob('/media/dmlab/My Passport/DATA/cross-domain/finetune_upper_limit_using_post-trained_raw/*_raw_k=1/classification_report.csv')
filepath_format = '/media/dmlab/My Passport/DATA/cross-domain/finetune_upper_limit_using_post-trained_raw/{}_{}_k={}/classification_report.csv'

records = []
for filepath in filepaths:
    domain = os.path.basename(os.path.dirname(filepath)).split('_')[0]
    raw_acc = '{:.2f}'.format(100*get_average_accuracy(domain, 'raw'))
    unk_acc = '{:.2f}'.format(100*get_average_accuracy(domain, 'unk'))

    records.append((domain, float(raw_acc), float(unk_acc)))

result_df = pd.DataFrame(records, columns = ['domain (kfold)', 'raw', 'unk'])
print(result_df.mean())
result_df

raw    81.7850
unk    80.5925
dtype: float64


,domain (kfold),raw,unk
0,books,80.83,81.99
1,dvd,83.22,79.66
2,electronics,77.00,79.19
3,kitchen,86.09,81.53
